#Sonification ακολουθιών DNA 

##Data dictionaries and Library imports

In [1]:
from music21 import *
import random

nucl2note = {'A' : "C", 'T' : "E", 'G' : "G", 'C' : "A"}

codons= {
    'GCT' : 'A', 'GCA' : 'A', 'GCG' : 'A', 'GCC' : 'A',
    'CGT' : 'R', 'CGC' : 'R', 'CGA': 'R', 'AGA': 'R', 'AGG': 'R', 'CGG': 'R',
    'AAT' : 'N', 'AAC' : 'N',
    'GAT' : 'D', 'GAC' : 'D',
    'TGT' : 'C', 'TGC' : 'C',
    'CAA' : 'Q', 'CAG' : 'Q',
    'GAA' : 'E', 'GAG' : 'E',
    'GGT' : 'G', 'GGC' : 'G', 'GGA' : 'G', 'GGG' : 'G', 
    'CAT' : 'H', 'CAC' : 'H',
    'ATT' : 'I', 'ATC' : 'I', 'ATA' : 'I',
    'CTT' : 'L', 'CTC' : 'L', 'CTA' : 'L', 'CTG' : 'L', 'TTA' : 'L', 'TTG' : 'L',
    'AAA' : 'K', 'AAG' : 'K',
    'ATG' : 'M',
    'TTT' : 'F', 'TTC' : 'F',
    'CCT' : 'P', 'CCC' : 'P', 'CCA' : 'P', 'CCG' : 'P',
    'TCT' : 'S', 'TCC' : 'S', 'TCA' : 'S', 'TCG' : 'S', 'AGT' : 'S', 'AGC' : 'S',
    'ACT' : 'T', 'ACC' : 'T', 'ACA' : 'T', 'ACG' : 'T',
    'TGG' : 'W',
    'TAT' : 'Y', 'TAC' : 'Y',
    'GTT' : 'V', 'GTC' : 'V', 'GTA' : 'V', 'GTG' : 'V',
    'TAA' : 'STOP', 'TGA' : 'STOP', 'TAG' : 'STOP'
}

codons2notes = { 'P' : 'G2', 'W' : 'C3', 'G' : 'F3', 'A' : 'G3', 'M' : 'C4', 'C' : 'F4', 'F' : 'G4', 'L' : 'C5', 'V' : 'F5', 'I' : 'G5',
                 'R' : 'E2', 'K' : 'D3', 'N' : 'E3', 'Q' : 'A3', 'H' : 'D4', 'Y' : 'E4', 'S' : 'A4', 'T' : 'E5' , 'D' : 'D5', 'E' : 'A5', 'STOP' : 'B4'
 }   


#1. DNA Sonification based on nucleotides

In [2]:
#Transmembrane Serine Protease 7 
#https://www.kaggle.com/nageshsingh/dna-sequence-dataset?select=example_dna.fa
TMPRSS7 = 'ATGTTTCGCATCACCAACATTGAGTTTCTTCCCGAATACCGACAAAAGGAGTCCAGGGAA\
TTTCTTTCAGTGTCACGGACTGTGCAGCAAGTGATAAACCTGGTTTATACAACATCTGCC\
TTCTCCAAATTTTATGAGCAGTCTGTTGTTGCAGATGTCAGCAACAACAAAGGCGGCCTC\
CTTGTCCACTTTTGGATTGTTTTTGTCATGCCACGTGCCAAAGGCCACATCTTCTGTGAA\
GACTGTGTTGCCGCCATCTTGAAGGACTCCATCCAGACAAGCATCATAAACCGGACCTCT\
GTGGGGAGCTTGCAGGGACTGGCTGTGGACATGGACTCTGTGGTACTAAATGAAGTCCTG\
GGGCTGACTCTCATTGTCTGGATTGACTGA'

def nucleotides_sonification(sequence, nucl2note, save_midi = 0, filename = 'nucleotides_sonification'):
    s = stream.Stream()
    notes = []
    for nucl in sequence:
        n = note.Note(nucl2note[nucl])#, quarterLength=0.25
        s.append(instrument.Piano())
        notes.append(n)
        s.append(n)
    if save_midi:
        s.write('midi', fp= str(filename) + '.midi')
    return notes

melody = nucleotides_sonification(TMPRSS7, nucl2note, 1)

#2. DNA Sonification based on triplets of Nucleotides

#2.1 Monophonic (Piano)

In [3]:
def triplet_nucleotide_sonification_mono(sequence, nucl2note, save_midi = 0, filename = 'triplet_nucleotide_sonification_mono'):
    notes = nucleotides_sonification(sequence, nucl2note)
    s = stream.Stream()
    start = end = 0
    size = len(sequence) - len(sequence)%3
    while end < size:
        end += 3
        c = chord.Chord([notes[start], notes[start + 1], notes[start + 2]])
        s.append(c)
        start = end
    if save_midi:
        s.write('midi', fp= str(filename) + '.midi')

triplet_nucleotide_sonification_mono(TMPRSS7, nucl2note, 1)

##2.2 Polyphonic (Piano, Guitar, Flute)

---



In [4]:
def triplet_nucleotide_sonification_poly(sequence, nucl2note, save_midi = 0, filename = 'triplet_nucleotide_sonification_poly'):
    notes = nucleotides_sonification(sequence, nucl2note)
    s = stream.Stream()
    p1 = stream.Part(id='part1')
    p2 = stream.Part(id='part2')
    p3 = stream.Part(id='part3')
    start = end = 0
    size = len(sequence) - len(sequence)%3
    while end < size:
        end += 3
        n1 = stream.Measure(number=1)
        n1.append(notes[start])
        p1.append(n1)

        n2 = stream.Measure(number=2)
        n2.append(notes[start + 1])
        p2.append(instrument.Guitar())
        p2.append(n2)

        n3 = stream.Measure(number=3)
        n3.append(notes[start + 2])
        p3.append(instrument.Flute())
        p3.append(n3)

        start = end
    s.insert(0, p1)
    s.insert(0, p2)
    s.insert(0, p3)
    s.recurse()
    if save_midi:
        s.write('midi', fp= str(filename) + '.midi')

triplet_nucleotide_sonification_poly(TMPRSS7, nucl2note, 1)



#3.  Monophonic (Piano) DNA Sonification based on codons (Without taking into consideration start codon as start point and end codons as end points)

In [5]:
def codon_sonification_mono(sequence, codons, codons2notes, save_midi = 0, filename = 'codon_sonification_mono'):
    notes = []
    s = stream.Stream()
    start = end = 0
    size = len(sequence) - len(sequence)%3
    while end < size:
        end += 3
        triplet = sequence[start:end]
        codon = codons[triplet]
        n = note.Note(codons2notes[codon])
        #s.append(instrument.Piano())
        s.append(n)
        notes.append(n)
        start = end
    if save_midi:
        s.write('midi', fp= str(filename) + '.midi')
    return notes


melody = codon_sonification_mono(TMPRSS7, codons, codons2notes, 1)

#4. Monophonic (Piano) DNA Sonification based on codons (Taking into consideration start codon as start point and end codons as end points)

In [6]:
def codon_sonification_mono2(sequence, codons, codons2notes, save_midi = 0, filename = 'codon_sonification_mono2'):
    s = stream.Stream()
    start = end = 0
    notes = []
    size = len(sequence) - len(sequence)%3
    flag = 0
    while end < size:
        end += 3
        triplet = sequence[start:end]
        codon = codons[triplet]
        if flag == 0:
            while codon != 'M' :
                n = note.Rest()
                notes.append(n)
                s.append(n)
                start += 1
                end += 1
                triplet = sequence[start:end]
                codon = codons[triplet]
                if  end > size:
                    n = note.Rest()
                    notes.append(n)
                    s.append(n)
                    if save_midi:
                        s.write('midi', fp= str(filename) + '.midi')
                    return notes
            flag = 1
            n = note.Note(codons2notes[codon])
            notes.append(n)
            #s.append(instrument.Piano())
            s.append(n)
        else:
            n = note.Note(codons2notes[codon])
            notes.append(n)
            #s.append(instrument.Piano())
            s.append(n)
            if codon == 'STOP':
                flag = 0
        start = end
    if save_midi:
        s.write('midi', fp= str(filename) + '.midi')
    return notes

melody = codon_sonification_mono2('AAACATGGTATAACCCATGTGATAAAAA', codons, codons2notes, 1)

#5. Polyphonic (Piano, Guitar, Flute) DNA Sonification based on codons

In [7]:
def codon_sonification_poly(sequence, codons, codons2notes, algorithm, save_midi = 0, filename = 'codon_sonification_poly'):

    notes1 = algorithm(sequence, codons, codons2notes)
    notes2 = algorithm(sequence[1:], codons, codons2notes)
    notes3 = algorithm(sequence[2:], codons, codons2notes)
    s = stream.Stream( )
    p1 = stream.Part(id='part1')
    for n in notes1:
        n1 = stream.Measure(number=1)
        n1.append(n)
        p1.append(n1)
    p1.append(note.Rest())
    p1.append(note.Rest())

    p2 = stream.Part(id='part2')
    n2 = stream.Measure(number=2)
    n2.append(note.Rest())
    p2.append(n2)
    for n in notes2:
        n2 = stream.Measure(number=2)
        n2.append(n)
        p2.append(instrument.Guitar())
        p2.append(n2)
    n2 = stream.Measure(number=2)
    n2.append(note.Rest())
    p2.append(n2)
        

    p3 = stream.Part(id='part3')
    n3 = stream.Measure(number=3)
    n3.append(note.Rest())
    n3.append(note.Rest())
    p3.append(n3)
    for n in notes3:
        n3 = stream.Measure(number=3)
        n3.append(n)
        p3.append(instrument.Flute())
        p3.append(n3)

    s.insert(0, p1)
    s.insert(0, p2)
    s.insert(0, p3)
    s.recurse()
    if save_midi:
        s.write('midi', fp= str(filename) + '.midi')
        
codon_sonification_poly(TMPRSS7, codons, codons2notes, codon_sonification_mono2, 1, 'codon_sonification_poly2')
codon_sonification_poly(TMPRSS7, codons, codons2notes, codon_sonification_mono2, 'codon_sonification_poly')

#Covid's DNA sonification

In [8]:
#https://www.kaggle.com/jamzing/sars-coronavirus-accession?select=SARS_CORONAVIRUS_NC_045512_sequence.fasta
covid = "...\
AAAAAAAAAAAAA"

In [10]:
codon_sonification_mono(covid, codons, codons2notes, 1, 'covid_mono')
codon_sonification_mono2(covid, codons, codons2notes, 1, 'covid_mono2')
melody = codon_sonification_poly(covid, codons, codons2notes, codon_sonification_mono, 1, 'covid_poly')